In [ ]:
import os
import pandas as pd

IND_FOLDER_PATH = "outputs/ind"
E2E_FOLDER_PATH = "outputs/e2e"

# Analyze individual tests

In [ ]:
csv_filenames = [f for f in os.listdir(IND_FOLDER_PATH) if f.endswith(".csv")]

# retrieve the output files
dataframes = []
for filename in csv_filenames:
    full_path = os.path.join(IND_FOLDER_PATH, filename)
    print(f"Processing individual outputs: {filename}")
    df = pd.read_csv(full_path)
    df["Agent"] = filename.split(".csv")[0]
    dataframes.append(df)
full_df = pd.concat(dataframes)

In [ ]:
categories = full_df.groupby(["Category", "Agent"])[["Pass Count", "Total Count"]].sum()
categories["Pass Percentage"] = categories["Pass Count"] / categories["Total Count"] * 100
pivot_categories = categories.pivot_table(index=["Category", "Total Count"], columns="Agent", values="Pass Percentage")
pivot_categories

In [ ]:
tasks = full_df.groupby(["Category", "Task", "Agent"])[["Pass Count", "Total Count"]].sum()
tasks["Pass Percentage"] = tasks["Pass Count"] / tasks["Total Count"] * 100
pivot_tasks = tasks.pivot_table(index=["Category", "Task", "Total Count"], columns="Agent", values="Pass Percentage")
pivot_tasks

# Analyze E2E tests